In [2]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import torch

from HJK.basis.monomials import monomials
from sklearn.preprocessing import PolynomialFeatures

#from deeptime.basis import Monomials
#import PySINDY.pysindy as ps


## Collect random datapoints from a domain

In [2]:
domain = 5
dim = 2
numIC = 1000

X = np.random.uniform(low=-domain, high=domain, size=(dim,numIC)).astype(float) 
np.shape(X)

(2, 1000)

## Monomials from `deeptime`

In [6]:
deeptime_lib = Monomials(p=2, d=2)
deeptime_lib.get_feature_names(["x1", "x2"])

['', 'x1', 'x2', 'x1^2', 'x1 x2', 'x2^2']

## Monomials from `pySINDy`

In [7]:
sindy_lib = ps.PolynomialLibrary(degree=2)

NameError: name 'ps' is not defined

## Monomials from d3s github

In [8]:
d3s_monomials = monomials(2)
d3s_monomials.display(np.array([1.0, 1.0, 1.0, 1.0]),2)

1.000001 + 1.00000 x_1 + 1.00000 x_2 + 1.00000 x_1^2


In [14]:
X = np.array([[1, 2], [1,3]])
obs = d3s_monomials(X)
np.shape(obs)
obs

array([[1., 1.],
       [1., 2.],
       [1., 3.],
       [1., 4.],
       [1., 6.],
       [1., 9.]])

In [13]:
Dobs = d3s_monomials.diff(X)
np.shape(obs)
obs

array([[[0., 0.],
        [0., 0.]],

       [[1., 1.],
        [0., 0.]],

       [[0., 0.],
        [1., 1.]],

       [[2., 4.],
        [0., 0.]],

       [[1., 3.],
        [1., 2.]],

       [[0., 0.],
        [2., 6.]]])

# Monomials from  `sklearn.PolynomialFeatures`

In [29]:
#X = torch.tensor([[1.0, 2.0], [1.0, 3.0]])
x = np.array([1.0,2.0], dtype=float)
monomial_basis = PolynomialFeatures(2)
monomial_basis.fit(X)
monomial_basis.get_feature_names(['x1','x2'])

['1', 'x1', 'x2', 'x1^2', 'x1 x2', 'x2^2']

In [30]:
Psi = monomial_basis.fit_transform(X)
Psi

array([[1., 1., 2., 1., 2., 4.],
       [1., 1., 3., 1., 3., 9.]])

In [31]:
def basis(x):
    monomial_basis = PolynomialFeatures(2)
    return monomial_basis.fit(x)

In [4]:
from OrthNet.orthnet import Poly

x = torch.DoubleTensor(np.random.random((10, 2)))
L = Poly(x, None, x, 2, recurrence = lambda x1, x2, : x1*x2)
print(L.tensor)

AttributeError: 'Tensor' object has no attribute 'get_dims'